# xAnzeige der Temperaturdaten aus der Heizung aus Temperaturen.db

In [103]:
import os, sys
from zoneinfo import ZoneInfo
import paramiko
import ipywidgets as widgets
import time
import datetime as dt
import sqlite3
import numpy as np
import warnings
from dotenv import dotenv_values
#import plotly.express as px   
import plotly.io as pio
pio.renderers.default='iframe'
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.plotting.backend = "plotly"
pd.options.mode.chained_assignment = None 

from scp import SCPClient
DB_FILENAME = 'Temperaturen.db'
DEFAULT_DURATION_DAYS=4
AGE_DIFF_SECONDS = 60*60*24

def createSSHClient(server, port, user, password):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())   
    client.connect(server, port, user, password)
    return client
    
def get_db_file():
    print('Hole Daten ...', end = '')
    config = dotenv_values(".env")
    ssh = createSSHClient("192.168.202.41", 22, config.get('TEMPDB_USER'), config.get('TEMPDB_PASSWD'))
    scp = SCPClient(ssh.get_transport())
    scp.get('/var/lib/grafana/' + DB_FILENAME)
    print('Feddisch')

def show_temperatur_file_status(db_filename, agediff):
    st=os.stat(db_filename)    
    mtime=st.st_mtime
    if time.time() - os.path.getmtime(db_filename) > agediff:
        button.style.button_color = '#FF0000'
        print('Die Temperaturdaten sind veraltet: ' + format(datetime.datetime.fromtimestamp(mtime)) )
    else:
        button.style.button_color = '#90ee90'
        print('Die Temperaturdaten sind aktuell: ' + format(datetime.datetime.fromtimestamp(mtime)) )

layout_btn = widgets.Layout(width='200px')
button = widgets.Button(description="Temperaturdaten holen", layout=layout_btn)
show_temperatur_file_status(DB_FILENAME, AGE_DIFF_SECONDS)

def on_button_clicked(b):
  with output:
    get_db_file()
    show_temperatur_file_status(DB_FILENAME, AGE_DIFF_SECONDS)

output = widgets.Output()

display(button, output)
button.on_click(on_button_clicked)

Die Temperaturdaten sind aktuell: 2023-10-26 11:57:40.128903


Button(description='Temperaturdaten holen', layout=Layout(width='200px'), style=ButtonStyle(button_color='#90e…

Output()

In [104]:
def get_df(db_filename):
    db_file = db_filename
    
    if os.path.exists(os.path.join('/var/lib/grafana', db_filename)):
        db_file = os.path.join('/var/lib/grafana', db_filename)
    elif os.path.exists(os.path.join('.', db_filename)):
        db_file = os.path.join('.', db_filename)    
    else:
        raise SystemExit('Die Datenbank ' + db_file + ' ist nicht vorhanden')
        
    try:
        con = sqlite3.connect(db_file)
        df = pd.read_sql_query("SELECT * from Temperaturen", con)
    except:
        raise SystemExit('Die Datenbank ' + db_file + ' ist nicht vorhanden')
    # Blanks mit np.nan ersetzen
    return df.replace(r'^\s*$', np.nan, regex=True)
    
def plot_dfplot(start, dauer, einheit):
    # Mit Dropdown: datetime.datetime
    # Mit DatePickier: datetime.date

    #start_object = datetime.datetime.strptime(start, '%Y-%m-%d')
    start_object = datetime.datetime(start.year, start.month, start.day)
    
    if einheit == 'Tage':
        end_date = start_object + datetime.timedelta(days=dauer)
    elif einheit == 'Stunden':
        end_date = start_object + datetime.timedelta(hours=dauer)
    else:
        end_date = start_object + datetime.timedelta(days=dauer)
    #print('Start: ' + str(start) + ' End: ' + str(end_date))
    
    # Hier nach Start und Enddatum filtern
    subset = df[(df['UnixTime'] > start_object ) &  (df['UnixTime'] < end_date )]
    subset.plot(x="UnixTime", y=["VorlaufTemp", "RuecklaufTemp", "WohnzimmerTemp", "AussenTemp"], grid=True)

In [106]:
def plot_plotly(start, dauer, einheit):
    # Mit Dropdown: datetime.datetime
    # Mit DatePickier: datetime.date

    start_object = datetime.datetime(start.year, start.month, start.day,  tzinfo=ZoneInfo("Europe/Berlin"))
    
    if einheit == 'Tage':
        end_object = start_object + datetime.timedelta(days=dauer)
    elif einheit == 'Stunden':
        end_object = start_object + datetime.timedelta(hours=dauer)
    else:
        end__object = start_object + datetime.timedelta(days=dauer)
    
    # Hier nach Start und Enddatum filtern
    subset = df[(df['UnixTime'] > start_object )]
    subset = subset[(subset['UnixTime'] <= end_object)]
    
    #display(subset['UnixTime'].min)
    #display(subset['UnixTime'].max)
    
    subset.set_index(['UnixTime'])
    subset = subset.astype({'WohnzimmerTemp':'float64'})
    subset = subset.astype({'AussenTemp':'float64'})    
    fig_title = "Temperaturverlauf Heizung von: " + str(start_object) + " bis: " + str(end_object)
    fig = subset.plot(x='UnixTime',y=['VorlaufTemp', 'RuecklaufTemp', 'WohnzimmerTemp', 'AussenTemp'], title=fig_title,
                     labels={
                     "UnixTime": "Zeit",
                     "VorlaufTemp": "VorlaufTemperatur",
                     "species": "Species of Iris"
                 },)
    fig.update_layout(xaxis_title="Zeit", yaxis_title="Temperaturen in °C", legend_title=None)    
    fig.show()
 
# main
try:    
    df= get_df(DB_FILENAME)
except SystemExit as _e:
    print(_e)
    sys.exit(-1)

# https://stackoverflow.com/questions/55449747/convert-column-of-epoch-timestamps-to-datetime-with-timezone
df['UnixTime'] = pd.to_datetime(df['UnixTime'], unit='s').dt.tz_localize('utc').dt.tz_convert('Europe/Berlin')
df.set_index(['UnixTime'])

#df.info()

widget_dauer = widgets.IntSlider(min=1,max=10,step=1, description="Dauer", value=DEFAULT_DURATION_DAYS)
#dates_us = df.UnixTime.dt.strftime('%Y-%m-%d').unique()
#widget_start = widgets.Select(options=np.sort(dates_us)[::-1], rows=1,description='Startdatum:')
widget_start_picker = widgets.DatePicker(description='Beginn Zeitraum', value = datetime.datetime.now().date() - datetime.timedelta(days=DEFAULT_DURATION_DAYS))
widget_einheit = widgets.Select(options=['Stunden','Tage'], rows=1, value='Tage', continous_update=False)
widgets.interact(plot_plotly, start= widget_start_picker, dauer = widget_dauer, einheit=widget_einheit);

interactive(children=(DatePicker(value=datetime.date(2023, 10, 22), description='Beginn Zeitraum', step=1), In…